In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
import numpy as np
import os
from keras import applications
from keras.optimizers import SGD
from keras import optimizers

from keras.layers import Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 100
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar100_trained_model_epoch1.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
def createModel(weights_path=None):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    if weights_path:
        model.load_weights(weights_path)

    return model

In [4]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Using real-time data augmentation.
Epoch 1/1
1563/1563 [==============================] - 414s 265ms/step - loss: 4.2714 - acc: 0.0514 - val_loss: 3.9203 - val_acc: 0.1154


In [5]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])

print('Test accuracy:', scores[1])

Saved trained model at /home/abhijeet/Project/saved_models/keras_cifar100_trained_model_epoch1.h5 
10000/10000 [==============================] - 19s 2ms/step
Test loss: 3.92033613968
Test accuracy: 0.1154


In [11]:
model.predict(x_train[0:1])

array([[ 0.00541702,  0.00277987,  0.00546529,  0.01433949,  0.01125209,
         0.01492464,  0.00709548,  0.01046569,  0.0124661 ,  0.01277787,
         0.00883815,  0.00814573,  0.00811636,  0.01905369,  0.00588235,
         0.02085546,  0.01123789,  0.02691448,  0.00829679,  0.01795888,
         0.00523904,  0.01444212,  0.01037069,  0.00276732,  0.00554867,
         0.01295475,  0.0068991 ,  0.01017849,  0.00336795,  0.01119657,
         0.00130839,  0.02645201,  0.01253734,  0.01115264,  0.00655399,
         0.00687936,  0.0054838 ,  0.02609635,  0.01178256,  0.00551217,
         0.00773937,  0.01604952,  0.00654221,  0.00808423,  0.00754214,
         0.00625899,  0.00706197,  0.01925762,  0.01195258,  0.00433882,
         0.00588741,  0.00898077,  0.0236123 ,  0.00221163,  0.00696301,
         0.00785017,  0.01294327,  0.01227016,  0.01020146,  0.01418163,
         0.01839999,  0.00441614,  0.00167953,  0.01077205,  0.00589135,
         0.00682825,  0.00728435,  0.00276679,  0.0

In [38]:
modelCopy = createModel('saved_models/keras_cifar100_trained_model.h5')

In [39]:
predictor = Sequential()
predictor.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:], weights = modelCopy.layers[0].get_weights()))
predictor.add(Activation('relu'))
predictor.add(Conv2D(32, (3, 3), weights=modelCopy.layers[2].get_weights()))
predictor.add(Activation('relu'))
predictor.add(MaxPooling2D(pool_size=(2, 2)))
predictor.add(Dropout(0.25))

predictor.add(Conv2D(64, (3, 3), padding='same', weights=modelCopy.layers[6].get_weights()))
predictor.add(Activation('relu'))
predictor.add(Conv2D(64, (3, 3), weights=modelCopy.layers[8].get_weights()))
predictor.add(Activation('relu'))
predictor.add(MaxPooling2D(pool_size=(2, 2)))
predictor.add(Dropout(0.25))

predictor.add(Flatten())
predictor.add(Dense(1000, weights=modelCopy.layers[13].get_weights()))
predictor.add(Activation('relu'))
# predictor created.

In [65]:
#len(predictor.predict(x_train[0:1])[0])
print(y_train[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [66]:
deepFeatures = []
respClass = []
for i in range(0,50000):
    deepFeatures.append(predictor.predict(x_train[i:i+1])[0])
    for j in range(0,100):
        if(y_train[i][j]==1):
            respClass.append(j)
            break


In [74]:
from sklearn.neighbors import NearestNeighbors


neigh = NearestNeighbors(n_neighbors=6)
neigh.fit(deepFeatures)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=6, p=2, radius=1.0)

In [ ]:
acc=[]
for i in range(0,50000):
    idxl = neigh.kneighbors([deepFeatures[i]], return_distance=False)
    ct=0
    idxl = idxl[0]
    #print("k")
    #print(respClass[i])
    for j in range(1,6):     
        #print(respClass[idxl[j]])
        if(respClass[idxl[j]]==respClass[i]):
            ct+=1
    acc.append(ct)

In [121]:
acc

[1, 5, 0, 3, 2, 0, 0, 2, 2, 3]

In [91]:
from sklearn.neighbors import NearestNeighbors

samples=[]
samples.append([1,1])
samples.append([2,2])
samples.append([3,3])
samples.append([4,4])
samples.append([5,5])
ne = NearestNeighbors(n_neighbors=3)
ne.fit(samples)
idx = ne.kneighbors([[2,2]], return_distance=False)

In [101]:
print(idx[0][0])

1
